In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as pl
import seaborn as sns
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv("../input/2015.csv")

# Shape of Data
> 158 rows × 12 columns

# Columns
- Country
- Region
- Happiness Rank
- Happiness Score
- Standard Error
- Economy (GDP per Capita)
- Family
- Health (Life Expectancy)
- Freedom
- Trust (Government Corruption)
- Generosity
- Dystopia Residual


In [ ]:
data.info()
data.describe()
data.columns

In [ ]:
data.head(10)

In [ ]:
'''
Now we will attempt to create a subplot of the scatter graphs of Happiness Scores and
the factors from column Economy GDP per capita to Generoristy to see which factors 
provide a postive corrolation and which factors provide a negative correlation 

'''

fig,axes= pl.subplots(1,len(data.columns.values)-6,figsize=(15,15))
for i,col in enumerate(data.columns.values[5:-1]):
    data.plot(x=[col],y=[data.columns[3]],kind="scatter",ax=axes[i])
pl.show()

corr_list=[]
for x in data.columns[5:-1]:
    corr_list.append(np.corrcoef(data[data.columns[3]],data[x])[1,0])
print(corr_list)

'''
As we can see, there is a acceptable postive correlations between happiness score
and the columns:
1. GDP per Capita
2.Family
3.Health[Life Expectancy]
4.Freedom

We will do further data analysis using the aforementioned Factors to help use determine
which factors have a strong positive impact on happiness.
'''
    


<h4>From the first 10 data, we can easily see that it is dominated by countries mostly from Western Europe followed by Australia and New Zealand. 
To further prove that,  let us plot the mean each of the region's happiness score
</h4>


In [ ]:
region_happiness = data.groupby('Region')['Happiness Score']
region_happiness.mean().plot.bar();
pl.ylabel('Happiness Score');

# Hypothesis one: 
> After seeing the above bar chart, we are going to assume that a country's average
happiness score is significantly affected by the region it is in. We will try to prove 
our assumption by carrying out more tests between theses two variable. We can ssume that someone
who lives in a more developed country will have a happier life. 
> To further prove our assumption we will plot a graph of Happiness Score vs GDP in the next cell

In [ ]:
print("Correlation strength between GDP per Capita and Happiness Score: ", np.corrcoef(data['Economy (GDP per Capita)'],data['Happiness Score'])[0,1])
gdpcorr = sns.regplot(data['Economy (GDP per Capita)'],data['Happiness Score'])



<h4>As we can see, there is mediocre positive correlation between the GDP per capita and
the Happiness of a country, with a pearson's correlation coefficent of 0.780965526866
To further prove our Hypothesis one:</h4>
- We will next plot a bar chart of the regions and the average GDP of the region.
- Create a scatter plot of mean happiness score and mean GDP per capita by region



In [ ]:
'''
Section for plotting the bar chart between average GDP per capita by region and region
'''
region_GDP = data.groupby('Region')['Economy (GDP per Capita)']
region_GDP.mean().plot.bar();
pl.ylabel('Economy (GDP per Capita)');


In [ ]:
pl.scatter(region_GDP.mean(), region_happiness.mean());
pl.xlabel("Mean GDP per Capita per Region");
pl.ylabel("Mean Happiness score per Region");


In [ ]:
np.corrcoef(region_GDP.mean(), region_happiness.mean())[0,1]

In [ ]:
'''
Now we will be exploring the relation ship between freedom and region 
'''
region_freedom=data.groupby('Region')[data.columns[8]]
region_freedom.mean().plot.bar()
pl.ylabel('Freedom')

'''
As we can see from the graphs below, the regions 
1.Australia and New Zealand
2.North America
3.SouthEastern Asia
4.Western Europe

have the highest freedom, Alsongdside this 3 out of 4 regions hppen to have the highest 
GDP per Capita, and Happiness Scores
'''


In [ ]:
'''
Now we will see the comparison between Regions and Health and Life Expectancy
'''

region_expectancy=data.groupby("Region")[data.columns[7]]
region_expectancy.mean().plot.bar()
pl.ylabel('Health and Life Expectancy')
'''
As we can see, Health and Life Expectancy are high in the following regions:
1.Australia and New Zealand
2.North America
3.Eastern Asia
4.Western Europe

This time We also see the same three countries having the same with Health and Life Expectancy
'''

In [ ]:
'''
Now we will see the comparison between Region and families
'''

region_family=data.groupby("Region")[data.columns[6]]
region_family.mean().plot.bar()
pl.ylabel("Mean Family Score")

'''
As we can see the same three regions have a strong postive family contribution.
'''

In [ ]:
'''
The conclusion that we can draw from the above data analysis is that the mean Happiness
of someone is greatly affected by the Region they live in.

More than anything else, If you live in Australia and New Zealand,North America or 
Western Europe, you are more likely to be happiest with a combination of factors such 
as 
1.Freedom
2. GDP per Capita
3. Health
4.Family


Although other Regions have high score in these categories, you need a combination of 
these factor to be able to be happy.
'''

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


In [ ]:
'''
Section for Random forest model building using the data set that we have
1. Covert Regions to dummy variables
'''
train=data.set_index("Country")
train = pd.get_dummies(train,prefix='Region')
train.head()



In [ ]:
#train["Estimated"]=train["Happiness Score"]+(train[train.columns[2]]*10)
labels=np.array(train["Happiness Score"])
print(labels)
train=train.drop(["Happiness Score"],1)
train=train.drop(["Happiness Rank"],1)
features_name=list(train.columns)
train=np.array(train)
print(features_name)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(train, labels, test_size = 0.25, random_state = 42)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
'''
Portion to Graph the output from the test data
'''

pl.scatter(predictions,test_labels)
pl.title("Test Vs Predictions with normal Regresstion Tree")
pl.xlabel("Predictons")
pl.ylabel("Test")
pl.show



In [ ]:
# New random forest with the regions
rf_region = RandomForestRegressor(n_estimators= 1000, random_state=42)
'''
'Region_Australia and New Zealand', 'Region_Central and Eastern Europe', 'Region_Eastern Asia', 
'Region_Latin America and Caribbean', 'Region_Middle East and Northern Africa', 'Region_North America', 
'Region_Southeastern Asia', 'Region_Southern Asia', 'Region_Sub-Saharan Africa', 'Region_Western Europe'
'''
region_indices = [features_name.index('Region_Australia and New Zealand'), 
                     features_name.index('Region_Central and Eastern Europe'),
                    features_name.index('Region_Eastern Asia'),
                     features_name.index('Region_Latin America and Caribbean'),
                     features_name.index('Region_Middle East and Northern Africa'),
                    features_name.index('Region_North America'),
                     features_name.index('Region_Southeastern Asia'),
                    features_name.index('Region_Southern Asia'),
                    features_name.index('Region_Sub-Saharan Africa'),
                    features_name.index('Region_Western Europe')]
train_region = train_features[:, region_indices]
test_region = test_features[:, region_indices]
# Train the random forest
rf_region.fit(train_region, train_labels)
# Make predictions and determine the error
predictions = rf_region.predict(test_region)
errors = abs(predictions - test_labels)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
'''
To Terry: Please add a regression line to all the scatter plots
'''

pl.scatter(predictions,test_labels)
pl.title("Test Vs Predictions with Regions Regresstion Tree")
pl.xlabel("Predictons")
pl.ylabel("Test")
pl.show
print(np.corrcoef(predictions,test_labels)[1,0])

In [ ]:
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features_name, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
x_values = list(range(len(importances)))
# Make a bar chart
pl.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
pl.xticks(x_values, features_name, rotation='vertical')
# Axis labels and title
pl.ylabel('Importance'); pl.xlabel('Variable'); pl.title('Variable Importances');


In [ ]:
'''
Conclusions we can draw from the random forest regressor 
Using the data that we have gathered from Random Forest Regressor, we have learned new 
things about the our original observations using the original data analysis.

1. The Greatest Factor that affects Happiness Score is GDP per capita,
2. Our origial assumption which claimed that Regions were the greatest factor 
in Happiness was actually disporved which is seen in the Variable importance Graph
3. Although the importance of the Regions werent deemed high, if we still train a 
model with the Regions as the factors, we still get an accuracy of 87.66% and a 
correlation of 0.796 between predicted and testing variables, 

4. Most suprising factor of all is that from the Features importance graph that we
    have,we can see the Economy is the greatest motivator of happiness while Freedom
    and Corruption perception arent that important which leads us to conclude that even
    a corrupt country can have a very happy population as long as GDP per Capita
    is high enough.
5. Whoever said money can't buy happiness should have looked at this analysis before
    making that assumption.


HOW THIS ANALYSIS HELPS THE WORLD:

Using this analysis of world happiness, People could determine what factors are important
to have the highest percieved happiness. Countries and leaders could determine what factors
should be taken into consideration when making decisons.

For example:
    The Candian Federal Government is in a dilemma regarding the Trans American Pipline
    which is a major pipline from Canada to the United States and generates a large amount
    of income i.e GDP for the Candian people. At the same time the First Nations tribes 
    claim that the Trans Mountain Pipeline violates their rights to their soverign land i.e 
    their freedom. Using the Conclusions that we have found in our data analysis the Canadian
    federal government can easily come to the conclusion with a very high confidenc that 
    if The Trans Mountain pipeline makes enough money, they can easily subvert the freedom of the First Nations people 
    and still keep a mojority of its citizens as happy or even happier and they are less likey to be voted
    out during elections

'''
